## Node 6

In [1]:
import numpy as np

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [2]:
class LogicGate:
        
    def __init__(self, gate_name, xdata, tdata):
        
        self.name = gate_name
        
        # 입력 데이터, 정답 데이터 초기화
        self.__xdata = xdata.reshape(4,2)  # 4개의 입력데이터 x1, x2 에 대하여 batch 처리 행렬
        self.__tdata = tdata.reshape(4,1)  # 4개의 입력데이터 x1, x2 에 대한 각각의 계산 값 행렬
        
        # 2층 hidden layer unit : 6개 가정,  가중치 W2, 바이어스 b2 초기화
        self.__W2 = np.random.rand(2,6)  # weight, 2 X 6 matrix
        self.__b2 = np.random.rand(6)
        
        # 3층 output layer unit : 1 개 , 가중치 W3, 바이어스 b3 초기화
        self.__W3 = np.random.rand(6,1)
        self.__b3 = np.random.rand(1)
                        
        # 학습률 learning rate 초기화
        self.__learning_rate = 1e-2
    
        print(self.name + " object is created")
            
    def feed_forward(self):        # feed forward 를 통하여 손실함수(cross-entropy) 값 계산
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.__xdata, self.__W2) + self.__b2  # 은닉층의 선형회귀 값
        a2 = sigmoid(z2)                                  # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3            # 출력층의 선형회귀 값
        y = a3 = sigmoid(z3)                              # 출력층의 출력
    
        # cross-entropy 
        return  -np.sum( self.__tdata*np.log(y + delta) + (1-self.__tdata)*np.log((1 - y)+delta ) )    
    
    def loss_val(self):          # 외부 출력을 위한 손실함수(cross-entropy) 값 계산 
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.__xdata, self.__W2) + self.__b2  # 은닉층의 선형회귀 값
        a2 = sigmoid(z2)                                  # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3            # 출력층의 선형회귀 값
        y = a3 = sigmoid(z3)                              # 출력층의 출력
    
        # cross-entropy 
        return  -np.sum( self.__tdata*np.log(y + delta) + (1-self.__tdata)*np.log((1 - y)+delta ) )
    
    
    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self):
        
        f = lambda x : self.feed_forward()
        
        print("Initial loss value = ", self.loss_val())
        
        for step in  range(10001):
            
            self.__W2 -= self.__learning_rate * numerical_derivative(f, self.__W2)
    
            self.__b2 -= self.__learning_rate * numerical_derivative(f, self.__b2)
        
            self.__W3 -= self.__learning_rate * numerical_derivative(f, self.__W3)
    
            self.__b3 -= self.__learning_rate * numerical_derivative(f, self.__b3)
    
            if (step % 400 == 0):
                print("\n*****************************************************************************")
                print("step = ", step, "  , loss value = ", self.loss_val())
                print("\nW2 = " , self.__W2, "\n")
                print("b2 = " , self.__b2, "\n")
                
    
    # query, 즉 미래 값 예측 함수
    def predict(self, xdata):
        
        z2 = np.dot(xdata, self.__W2) + self.__b2         # 은닉층의 선형회귀 값
        a2 = sigmoid(z2)                                  # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3            # 출력층의 선형회귀 값
        y = a3 = sigmoid(z3)                              # 출력층의 출력
    
        if y > 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result

In [3]:
# AND Gate 객체 생성 및 training

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 0, 0, 1])

and_obj = LogicGate("AND", xdata, tdata)

and_obj.train()

AND object is created
Initial loss value =  7.857415836115302

*****************************************************************************
step =  0   , loss value =  7.560540411791716

W2 =  [[0.27131878 0.04431251 0.92402048 0.66427315 0.49256973 0.91603111]
 [0.60041714 0.4946732  0.62878787 0.44852881 0.60840316 0.95246285]] 

b2 =  [0.63829351 0.82428735 0.13108985 0.16549603 0.63902744 0.39416276] 


*****************************************************************************
step =  400   , loss value =  2.1580494605277254

W2 =  [[0.23254104 0.01078072 0.97669124 0.66706558 0.44648655 0.90650917]
 [0.55622934 0.46072039 0.67031727 0.44972529 0.55560916 0.94430708]] 

b2 =  [ 0.64984914  0.78828139 -0.14577841  0.09485076  0.72906417  0.31565178] 


*****************************************************************************
step =  800   , loss value =  2.007731224751578

W2 =  [[ 0.192256   -0.03975745  1.11328742  0.72227519  0.39200984  0.92163477]
 [ 0.50584942  0.39709


*****************************************************************************
step =  8800   , loss value =  0.06021103605621456

W2 =  [[-0.63862705 -1.56074909  3.02781538  1.5126424  -0.47675855  1.54341864]
 [-0.47937669 -1.40032676  3.07786956  1.43214466 -0.47516021  1.76843146]] 

b2 =  [ 1.00041393  1.89057108 -4.4147122  -1.84747239  1.21979934 -2.17111993] 


*****************************************************************************
step =  9200   , loss value =  0.055668643992607014

W2 =  [[-0.65534343 -1.57689631  3.05302247  1.52455182 -0.49619192  1.55906615]
 [-0.49734911 -1.41919233  3.10264997  1.44550488 -0.49509554  1.78260997]] 

b2 =  [ 1.00547042  1.92233182 -4.45458834 -1.87331947  1.22180632 -2.19958538] 


*****************************************************************************
step =  9600   , loss value =  0.051696116679843925

W2 =  [[-0.67094765 -1.59202735  3.07646277  1.53574958 -0.51448757  1.57377771]
 [-0.51411024 -1.43687985  3.1256722   1.4

In [4]:
# AND Gate prediction

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for data in test_data:
    print(and_obj.predict(data))

(array([0.00021671]), 0)
(array([0.01270217]), 0)
(array([0.01298618]), 0)
(array([0.97811597]), 1)


In [5]:
# OR Gate 객체 생성 및 training

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 1])

or_obj = LogicGate("OR", xdata, tdata)

or_obj.train()

OR object is created
Initial loss value =  2.7524555085641276

*****************************************************************************
step =  0   , loss value =  2.7348391986675304

W2 =  [[0.90551382 0.07057456 0.26304294 0.95993983 0.99968297 0.46228942]
 [0.17499208 0.70608473 0.2069516  0.51666071 0.03815778 0.49789214]] 

b2 =  [0.55017454 0.70485939 0.60068648 0.01724031 0.55682268 0.01332848] 


*****************************************************************************
step =  400   , loss value =  2.0178144973705043

W2 =  [[1.00889639 0.12012986 0.30001267 0.97259969 1.00473241 0.68038624]
 [0.30548604 0.74842845 0.24520009 0.53108662 0.04501442 0.71990472]] 

b2 =  [ 0.38373587  0.64041409  0.55554984  0.0047772   0.54007201 -0.16940006] 


*****************************************************************************
step =  800   , loss value =  1.8045495850506694

W2 =  [[1.14040755 0.15890271 0.30383875 1.05945169 1.01232469 0.97287678]
 [0.47617026 0.78196958 0.


*****************************************************************************
step =  9200   , loss value =  0.04425187498919364

W2 =  [[ 2.6723061  -0.00430259 -0.5920378   2.6363073   0.83851847  3.67993386]
 [ 2.43276164  0.65398563 -0.72812864  2.50401813 -0.23024199  3.79901996]] 

b2 =  [-1.36330915  0.78052419  0.95295604 -1.37771179  0.79429857 -1.95367758] 


*****************************************************************************
step =  9600   , loss value =  0.04140100796897924

W2 =  [[ 2.69250186 -0.0097076  -0.61010475  2.65704677  0.83436666  3.70599096]
 [ 2.45567945  0.65000888 -0.74652859  2.52626651 -0.23701632  3.82298563]] 

b2 =  [-1.37474814  0.78574078  0.95670694 -1.38874556  0.80041545 -1.96567478] 


*****************************************************************************
step =  10000   , loss value =  0.03886550643333756

W2 =  [[ 2.71155898 -0.01487184 -0.62732296  2.67661209  0.83041274  3.73042831]
 [ 2.47724284  0.6462219  -0.76401117  2.54

In [6]:
# OR Gate prediction

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for data in test_data:
    print(or_obj.predict(data))

(array([0.02214897]), 0)
(array([0.99212691]), 1)
(array([0.99210227]), 1)
(array([0.99936556]), 1)


In [7]:
# NAND Gate 객체 생성 및 training

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([1, 1, 1, 0])

nand_obj = LogicGate("NAND", xdata, tdata)

nand_obj.train()

NAND object is created
Initial loss value =  3.077560274526884

*****************************************************************************
step =  0   , loss value =  3.048820861899971

W2 =  [[0.44977019 0.28420476 0.10718483 0.87120376 0.98653187 0.17172484]
 [0.32132013 0.35168131 0.99190974 0.08737872 0.5288921  0.82498159]] 

b2 =  [0.32130911 0.77692842 0.68869149 0.78345068 0.78156766 0.64534591] 


*****************************************************************************
step =  400   , loss value =  2.251911879301405

W2 =  [[0.539277   0.27709852 0.15012547 0.80152438 0.90315809 0.10947303]
 [0.40417845 0.34459724 1.05770305 0.03700965 0.47351467 0.7509013 ]] 

b2 =  [0.27191197 0.77187355 0.61569674 0.81678116 0.91675163 0.67913214] 


*****************************************************************************
step =  800   , loss value =  2.1793592644434057

W2 =  [[0.66456781 0.27347855 0.204648   0.75518527 0.84876845 0.07434103]
 [0.51621152 0.34108753 1.1497204


*****************************************************************************
step =  8800   , loss value =  0.07854008303327778

W2 =  [[ 3.15630398 -0.36737541  2.32642917 -0.05177899  0.4606313  -0.48859334]
 [ 2.88224514 -0.2541019   2.66579296 -0.51001577  0.28429702  0.04807362]] 

b2 =  [-4.33111168  0.96811211 -3.51160658  1.12620704  1.80281436  0.95003096] 


*****************************************************************************
step =  9200   , loss value =  0.07215756897624007

W2 =  [[ 3.18344993 -0.38424085  2.36332708 -0.07018523  0.45730623 -0.50224542]
 [ 2.92055156 -0.27118153  2.68851757 -0.52492318  0.28238099  0.03182879]] 

b2 =  [-4.38400379  0.97072567 -3.5613781   1.12781394  1.80960457  0.9522889 ] 


*****************************************************************************
step =  9600   , loss value =  0.06663529325118461

W2 =  [[ 3.20865665 -0.40016815  2.39735765 -0.08760962  0.45422371 -0.51519234]
 [ 2.95587289 -0.28733199  2.70983204 -0.539

In [8]:
# NAND Gate prediction

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for data in test_data:
    print(nand_obj.predict(data))

(array([0.99895072]), 1)
(array([0.98388541]), 1)
(array([0.98419633]), 1)
(array([0.02818861]), 0)


In [9]:
# XOR Gate 객체 생성

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 0])


xor_obj = LogicGate("XOR", xdata, tdata)

xor_obj.train()

XOR object is created
Initial loss value =  4.719492911419669

*****************************************************************************
step =  0   , loss value =  4.62576643873056

W2 =  [[0.56513138 0.64985257 0.77769655 0.02929235 0.28361313 0.24628184]
 [0.22644505 0.36826693 0.4148508  0.47443348 0.08181696 0.79594833]] 

b2 =  [0.07151767 0.09746544 0.48548136 0.81132402 0.57809749 0.13768366] 


*****************************************************************************
step =  400   , loss value =  2.7765722603826277

W2 =  [[0.55158193 0.64076033 0.76541389 0.02686919 0.27573738 0.22431077]
 [0.21760585 0.35621584 0.39238373 0.45527926 0.07218529 0.78916566]] 

b2 =  [0.0341333  0.08661174 0.48328521 0.76451813 0.56405573 0.13951978] 


*****************************************************************************
step =  800   , loss value =  2.775368482063537

W2 =  [[0.55664347 0.63538205 0.75489275 0.04355977 0.27438153 0.19887838]
 [0.22734953 0.34764739 0.3720085  


*****************************************************************************
step =  9200   , loss value =  2.6773862184150374

W2 =  [[ 0.78689778  0.63703845  1.47938948  0.26761112  0.4321799  -0.16609545]
 [ 0.63303694  0.35387792  1.39611205  0.53246205 -0.12889063  0.8631299 ]] 

b2 =  [0.19786014 0.08699769 0.44229921 0.76595379 0.59743959 0.18892282] 


*****************************************************************************
step =  9600   , loss value =  2.6378318045692386

W2 =  [[ 0.80748774  0.63632268  1.66168213  0.26852304  0.47703118 -0.1846697 ]
 [ 0.66421591  0.34740977  1.60439531  0.53285211 -0.1451187   0.90423666]] 

b2 =  [0.2130017  0.08280055 0.36461658 0.76609924 0.603917   0.20709918] 


*****************************************************************************
step =  10000   , loss value =  2.5823894030397563

W2 =  [[ 0.82668577  0.63722285  1.87642975  0.26736595  0.5345429  -0.20391735]
 [ 0.69320874  0.3394574   1.84271888  0.53247445 -0.16215

In [10]:
test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for data in test_data:
    print(xor_obj.predict(data))

(array([0.40671468]), 0)
(array([0.51879867]), 1)
(array([0.54015848]), 1)
(array([0.54532765]), 1)
